In [94]:
# from sympy import *
import numpy as np
import shapely.wkt

from functions import find_z

# %load_ext autoreload
%autoreload 2

In [95]:

# random area: 
# wkt = 'POLYGON((117813.6 487266.1,117813.6 487370.7,117942.1 487370.7,117942.1 487266.1,117813.6 487266.1))'

# small area: 
# wkt = 'POLYGON((117813 487266,117813 487270,117815 487270,117815 487266,117813 487266))' # small! 

# portal:
# wkt = 'POLYGON((118032.6 488782.8,118032.6 488793.5,118078.8 488793.5,118078.8 488782.8,118032.6 488782.8))' # portal

# portal zonder palen
# wkt = 'POLYGON((118037.0 488782.8,118037.0 488793.5,118072.0 488793.5,118072.0 488782.8,118037.0 488782.8))' # portal zonder palen

# brug
wkt = 'POLYGON((117875.3 487304,117875.3 487329,117906.1 487329,117906.1 487304,117875.3 487304))' # brug

# stukje brug
# wkt = 'POLYGON((117890.1 487311,117890.1 487312.5,117892 487312.5,117892 487311,117890.1 487311))' # brug




In [96]:
# bij "brug"
# wkt = 'POLYGON((117875.3 487304,117875.3 487329,117906.1 487329,117906.1 487304,117875.3 487304))' # brug

models = {'2018-1': [[1.18046284e+05, 4.88788266e+05, 7.32619906e+00], # point
                      [-0.01907583, -0.00597143 , 0.99980021]],         # normal
          '2019-1': [[1.18043622e+05, 4.88788105e+05, 7.28090065e+00], 
                      [-0.0191627,  -0.00399964 , 0.99980838]],
          '2018-2': [[1.18067443e+05, 4.88788088e+05, 7.32255846e+00], 
                      [ 0.01882162 ,-0.00257657,0.99981954]],
          '2019-2': [[1.18067129e+05, 4.88788024e+05, 7.33989558e+00], 
                      [-0.01913283 , 0.00412501 ,-0.99980844]]}

In [61]:

p1, p2, p3 = [1.18046284e+05, 4.88788266e+05, 7.32619906e+00]
r, s, t = v1, v2, v3 = [-0.01907583, -0.00597143 , 0.99980021]

normal = [v1,v2,v3] 
point = [p1,p2,p3]


In [ ]:
polygon = shapely.wkt.loads(wkt)
bottom_left, bottom_right, upper_right, upper_left, _ = list(polygon.exterior.coords)
# corners = ['bottom_left', 'bottom_right', 'upper_right', 'upper_left']
corners = ['bl','br','ur','ul']

In [84]:
z_vals = {}

for key in models:
    point = models[key][0]
    normal = models[key][1]
    
    # normal[2] = abs(normal[2])
    centroid = polygon.centroid
    x,y = centroid.x, centroid.y
    z = find_z(point, normal, x, y)
    z_vals[f'{key}'] = (x,y,z)
    print(f'{key} \t {x}\t {y}\t {z} \t{normal[2]}')
    
    '''
    for i, coords in enumerate(polygon.exterior.centroid):
        if i == 4:
            break

        x,y = coords
        z = find_z(point, normal, x, y)
        
        z_vals[f'{key}-{corners[i]}'] = (x,y,z)
        print(f'{key} \t {corners[i]}\t {x}\t {y}\t {z} \t{normal[2]}')
        
        '''
    
    

2018-1 	 117890.70000000001	 487316.5	 -4.432591808928383 	0.99980021
2019-1 	 117890.70000000001	 487316.5	 -1.5370776826077577 	0.99980838
2018-2 	 117890.70000000001	 487316.5	 6.857414610640649 	0.99981954
2019-2 	 117890.70000000001	 487316.5	 4.644914178192723 	-0.99980844


In [ ]:
z_diffs = {}

# 2018
for corner in corners:
    key1 = f'2018-1-{corner}'
    key2 = f'2018-2-{corner}'
    
    z_diffs[f'{corner}_2018'] = z_vals[key2][2] - z_vals[key1][2]
    
    
# 2018
for corner in corners:
    key1 = f'2019-1-{corner}'
    key2 = f'2019-2-{corner}'
    
    z_diffs[f'{corner}_2019'] = z_vals[key2][2] - z_vals[key1][2]
  

In [88]:
for jaar in [2018, 2019]:
    z_diffs[f'{jaar}'] = z_vals[f'{jaar}-1'][2] - z_vals[f'{jaar}-2'][2]
    
z_diffs

{'bl_2018': 11.916183022200023,
 'br_2018': 11.831293316733843,
 'ur_2018': 10.663829816938497,
 'ul_2018': 10.748719522404675,
 'bl_2019': 11.673992354465172,
 'br_2019': 11.470837182059745,
 'ur_2019': 11.469916974365308,
 'ul_2019': 11.673072146770737,
 '2018': -11.290006419569032,
 '2019': -6.181991860800481}

In [55]:
difference = 0
print(f'corner \t 2018 \t 2019 \t difference')
for corner in corners:
    key1 = f'{corner}_2018'
    key2 = f'{corner}_2019'
    difference = z_diffs[key1] - z_diffs[key2]
    print(f'{corner} \t {z_diffs[key1]} \t {z_diffs[key2]} \t {difference}')
    
    difference += difference
    
print(difference / 4 )
    

corner 	 2018 	 2019 	 difference
bl 	 11.916183022200023 	 11.673992354465172 	 0.24219066773485132
br 	 11.831293316733843 	 11.470837182059745 	 0.36045613467409865
ur 	 10.663829816938497 	 11.469916974365308 	 -0.8060871574268109
ul 	 10.748719522404675 	 11.673072146770737 	 -0.9243526243660618
-0.4621763121830309


In [23]:

find_z(models['2018-1'][0], models['2018-1'][1], 
       models['2018-1'][0][0], models['2018-1'][0][1])  # 7.2809006499996

7.326199059999593

In [22]:
models['2019-1']

[[118043.622, 488788.105, 7.28090065], [-0.0191627, -0.00399964, 0.99980838]]

In [32]:
abs(-3)

3

In [74]:
float(np.array(3.00000000001))

3.00000000001